# 1. Import depedencies


In [35]:
#%pip install stable-baselines3[extra]

In [4]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment


In [68]:
env_name = 'CartPole-v0'
env= gym.make(env_name, render_mode="human")

In [59]:
episodes= 5
for episode in range(1, episodes+1):
    state= env.reset()
    done= False
    score=0
    
    while not done:
        env.render()
        action= env.action_space.sample() 
        n_state, reward, done, truncated, info= env.step(action)
        score+= reward
    print(f'Episode: {episode}, Score: {score}')
env.close() 

Episode: 1, Score: 19.0
Episode: 2, Score: 24.0
Episode: 3, Score: 12.0
Episode: 4, Score: 20.0
Episode: 5, Score: 16.0


## Understanding The ENV


In [43]:
env.action_space

Discrete(2)

In [45]:
env.observation_space.sample()

array([-1.7439504e+00, -2.3050178e+38, -1.7552522e-01,  1.9505366e+38],
      dtype=float32)

In [46]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [47]:
env.observation_space.sample()

array([3.9216175e+00, 2.6617539e+38, 3.6721665e-01, 1.5634062e+37],
      dtype=float32)

# 3. Train an RL Model


In [5]:
log_path= os.path.join('Training', 'Logs')

In [4]:
log_path

'Training/Logs'

In [6]:
env_name = 'CartPole-v0'
env= gym.make(env_name, render_mode="human")
env= DummyVecEnv([lambda: env])
model= PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

/home/iticbcn/Escritorio/CE/IA/Virtualsenv/RL/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/home/iticbcn/Escritorio/CE/IA/Virtualsenv/RL/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device


/home/iticbcn/Escritorio/CE/IA/Virtualsenv/RL/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [16]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_8
-----------------------------
| time/              |      |
|    fps             | 47   |
|    iterations      | 1    |
|    time_elapsed    | 43   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 47          |
|    iterations           | 2           |
|    time_elapsed         | 86          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008288235 |
|    clip_fraction        | 0.0883      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00567    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.31        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0135     |
|    value_loss           | 51.1        |
-----------------------------------------
---

# 4. Save and Reload Model


In [4]:
PPO_path=os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [18]:
model.save(PPO_path)

/home/iticbcn/Escritorio/CE/IA/Virtualsenv/RL/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [ ]:
#del model

In [5]:
model= PPO.load(PPO_path, env=env)

# 5. Evaluation


In [65]:
evaluate_policy(model,env, n_eval_episodes=10, render=True)

(np.float64(200.0), np.float64(0.0))

In [66]:
env.close()

# 6. Test Model


In [7]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()  # Esto debería devolver una sola observación
    done = False
    score = 0

    while not done:
        env.render()
        # print(episode,obs)
        # print(model.predict(obs))
        action, _ = model.predict(obs)
        # print(action)
        obs, reward, done, info = env.step(action)
        score += reward
    print(f'Episode: {episode}, Score: {score}')

Episode: 1, Score: [200.]
Episode: 2, Score: [200.]
Episode: 3, Score: [200.]
Episode: 4, Score: [200.]
Episode: 5, Score: [200.]


In [14]:
env.close()

In [13]:
obs
action, _ = model.predict(obs)
env.action_space.sample()
env.step(action)

(array([[ 0.00069122, -0.17810076,  0.04713419,  0.26687437]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

# 7. Viewing Logs in Tensorboard


In [25]:
training_log_path=os.path.join(log_path, 'PPO_2')    

In [26]:
#load_ext tensorboard
%tensorboard --logdir={training_log_path}
#http://localhost:6008/?darkMode=true#timeseries

Reusing TensorBoard on port 6006 (pid 17342), started 0:11:14 ago. (Use '!kill 17342' to kill it.)

In [28]:
!^C
!kill 17342

/bin/bash: línea 1: ^C: orden no encontrada


# 8. Adding a callback to the training Stage


In [7]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [8]:
save_path=os.path.join('Training', 'Saved Models')

In [10]:
stop_callback= StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback= EvalCallback(env, 
                        callback_on_new_best=stop_callback, 
                        eval_freq=10000, 
                        best_model_save_path=save_path,
                        verbose=1)

In [11]:
model= PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


/home/iticbcn/Escritorio/CE/IA/Virtualsenv/RL/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [13]:
model.learn(total_timesteps=20000, callback= eval_callback)

Logging to Training/Logs/PPO_10
-----------------------------
| time/              |      |
|    fps             | 47   |
|    iterations      | 1    |
|    time_elapsed    | 42   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 47           |
|    iterations           | 2            |
|    time_elapsed         | 86           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0077729737 |
|    clip_fraction        | 0.0843       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | 7.46e-05     |
|    learning_rate        | 0.0003       |
|    loss                 | 7.7          |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0125      |
|    value_loss           | 54.7         |
---------------------------

/home/iticbcn/Escritorio/CE/IA/Virtualsenv/RL/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=188.20 +/- 12.69
Episode length: 188.20 +/- 12.69
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 188          |
|    mean_reward          | 188          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0071393196 |
|    clip_fraction        | 0.0779       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.598       |
|    explained_variance   | 0.259        |
|    learning_rate        | 0.0003       |
|    loss                 | 24.6         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.016       |
|    value_loss           | 66.3         |
------------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 43    |
|    iterations     

# 9. Changing Policies


In [ ]:
net_arch=[dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [ ]:
model= PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback= eval_callback)

# 10. Using an Alternate Algorithm


In [ ]:
from stable_baselines3 import DQN

In [ ]:
model= DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback= eval_callback)